In [24]:
#Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

In [25]:
#Create File Paths
mouse_metadata = pd.read_csv('data/Mouse_metadata.csv')
study_results = pd.read_csv('data/Study_results.csv')

In [28]:
#Merge two tables into one based on mouse ID
merged_mouse_data = pd.merge(study_results, mouse_metadata, on="Mouse ID")
merged_mouse_data.set_index('Mouse ID').head()

,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
Mouse ID,,,,,,,
b128,0,45.000000,0,Capomulin,Female,9,22
b128,5,45.651331,0,Capomulin,Female,9,22
b128,10,43.270852,0,Capomulin,Female,9,22
b128,15,43.784893,0,Capomulin,Female,9,22
b128,20,42.731552,0,Capomulin,Female,9,22


In [29]:
#Check number of mice
merged_mouse_data['Mouse ID'].nunique()

249

In [36]:
#Find the duplicate mice by ID number that shows up for Mouse ID and Timepoint
#Optional: Get all the data for the duplicate mouse ID. 
duplicate_mouse = merged_mouse_data[merged_mouse_data.duplicated(subset=['Mouse ID', 'Timepoint'], keep=False)]
duplicate_mouse.set_index('Mouse ID')

,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
Mouse ID,,,,,,,
g989,0,45.000000,0,Propriva,Female,21,26
g989,0,45.000000,0,Propriva,Female,21,26
g989,5,48.786801,0,Propriva,Female,21,26
g989,5,47.570392,0,Propriva,Female,21,26
g989,10,51.745156,0,Propriva,Female,21,26
g989,10,49.880528,0,Propriva,Female,21,26
g989,15,51.325852,1,Propriva,Female,21,26
g989,15,53.442020,0,Propriva,Female,21,26
g989,20,55.326122,1,Propriva,Female,21,26


In [38]:
#Create a clean DataFrame by dropping the duplicate mouse by its ID.
#Dropping all data for Mouse ID g989 due to duplicated Timepoints that have differing Tumor Volume values. If Tumor Volume values 
#were same at each Timepoint, I would remove only the duplicated rows.

clean_mouse_data = merged_mouse_data[merged_mouse_data['Mouse ID'] != 'g989']
clean_mouse_data.set_index('Mouse ID').head()

,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
Mouse ID,,,,,,,
b128,0,45.000000,0,Capomulin,Female,9,22
b128,5,45.651331,0,Capomulin,Female,9,22
b128,10,43.270852,0,Capomulin,Female,9,22
b128,15,43.784893,0,Capomulin,Female,9,22
b128,20,42.731552,0,Capomulin,Female,9,22


In [39]:
#Recheck the number of mice in the clean DataFrame.

clean_mouse_data['Mouse ID'].nunique()

248

In [ ]:
#Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

In [ ]:
#Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas

In [ ]:
#Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.

In [ ]:
#Generate a pie plot showing the distribution of female versus male mice using pandas

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)

# Create empty list to fill with tumor vol data (for plotting)

# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes

    
    # add subset 
  
    
    # Determine outliers using upper and lower bounds

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest

In [ ]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin

In [ ]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen